In [ ]:
import time
import requests
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# --- Configuration ---
API_KEY = '' 
API_ENDPOINT = 'https://svcs.ebay.com/services/search/FindingService/v1'
REFERENCE_FILE = './output/prismatic_evolutions.csv'     # This should have columns such as 'item' and 'max_price'
COSINE_SIM_THRESHOLD = 0.5           # Adjust this threshold as needed; 1.0 means identical

# --- Load the reference file ---
reference_df = pd.read_csv(REFERENCE_FILE)  # Expecting columns: item, max_price


# --- Function: Search eBay API ---
def search_ebay(query, page=1):
    """
    Makes a request to the eBay Finding API using a given query.
    The API call filters for fixed-price ("Buy It Now") listings.
    """
    headers = {
        'X-EBAY-SOA-SECURITY-APPNAME': API_KEY,
        'X-EBAY-SOA-OPERATION-NAME': 'findItemsByKeywords',
        'X-EBAY-SOA-RESPONSE-DATA-FORMAT': 'JSON'
    }
    
    params = {
        'keywords': query,
        'paginationInput.entriesPerPage': 10,
        'paginationInput.pageNumber': page,
        # Filter for fixed price items (typically Buy It Now)
        'itemFilter(0).name': 'ListingType',
        'itemFilter(0).value': 'FixedPrice'
    }
    
    response = requests.get(API_ENDPOINT, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print('Error fetching data from eBay API:', response.status_code)
        return None


# --- Function: Process Results ---
def process_results(reference_item, max_price, results):
    """
    Process the JSON response from eBay.
    
    1. Extract listing titles and prices.
    2. Apply cosine similarity between the reference item text and the listing titles.
    3. Return the ones under the max_price and above the similarity threshold.
    """
    matched_items = []
    
    try:
        # Navigate the API response JSON structure (may need adjustment per eBay API documentation)
        items = results['findItemsByKeywordsResponse'][0]['searchResult'][0]['item']
    except KeyError:
        items = []
    
    if not items:
        return matched_items
    
    # Extract titles from each result.
    titles = [item['title'][0] for item in items if 'title' in item]
    
    # Use TF-IDF Vectorizer for cosine similarity matching.
    # The vectorizer is fitted on both the reference text and the found titles.
    vectorizer = TfidfVectorizer().fit([reference_item] + titles)
    reference_vec = vectorizer.transform([reference_item])
    title_vecs = vectorizer.transform(titles)
    
    similarities = cosine_similarity(reference_vec, title_vecs).flatten()
    
    # Iterate through each result, check the price and computed cosine similarity.
    for idx, item in enumerate(items):
        try:
            # Parsing the price; the key may vary based on API version and response format.
            price = float(item['sellingStatus'][0]['currentPrice'][0]['__value__'])
        except Exception as e:
            print("Error parsing price:", e)
            price = float('inf')
        
        if price <= max_price and similarities[idx] >= COSINE_SIM_THRESHOLD:
            matched_items.append({
                'title': item['title'][0],
                'price': price,
                'similarity': similarities[idx],
                'itemId': item['itemId'][0]
            })
    
    return matched_items


# --- Function: Scan Items ---
def scan_items():
    """
    Iterate over each reference item, perform a search via the eBay API,
    filter the results using price and similarity, and collect any matches.
    """
    alerts = []
    for index, row in reference_df.iterrows():
        reference_item = row['item']
        try:
            max_price = float(row['max_price'])
        except ValueError:
            print(f"Invalid max_price for item {reference_item}")
            continue
        
        print(f"Searching for: {reference_item}")
        results = search_ebay(reference_item)
        if results:
            matches = process_results(reference_item, max_price, results)
            if matches:
                print(f"Found matches for {reference_item}: {matches}")
                alerts.extend(matches)
    return alerts


In [ ]:
# --- Main Loop ---
if __name__ == "__main__":
    while True:
        alerts = scan_items()
        if alerts:
            # Here you can incorporate an alert system: send an email, SMS, etc.
            print("Alert! Matching items found:")
            for alert in alerts:
                print(alert)
        else:
            print("No matching items found in this scan.")
        # Pause for a minute before scanning again
        time.sleep(60)